<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileCKD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
DrugDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/WideTable/DrugDF.xlsx')
DrugDF.shape

(52619, 28)

In [3]:
import pandas as pd
DiagDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/WideTable/DiagDF.xlsx')
DiagDF.shape

(52619, 15)

In [4]:
import pandas as pd
LabDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/WideTable/LabDF.xlsx')
LabDF.shape

(52619, 34)

In [5]:
import pandas as pd
VitalDF = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/WideTable/VitalDF.xlsx')
VitalDF.shape

(52619, 8)

In [6]:
VitalDF = VitalDF.iloc[:, 1:]
VitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
0,2299,5812902,-7.158596,138,76,20.72,1.38
1,2299,6947220,5.550691,134,66,20.27,1.36
2,2299,8186767,-6.063710,103,57,19.05,1.33
3,2299,9385133,-12.812245,128,62,NaN,NaN
4,2304,4916314,28.287500,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,147,87,NaN,NaN
52615,3319971,10553916,-4.256753,143,86,NaN,NaN
52616,3320048,10572126,-0.692078,131,79,NaN,NaN
52617,3320050,10572565,7.217875,138,NaN,NaN,NaN


In [7]:
LabDF = LabDF.iloc[:, 1:]
LabDF

,PatientUID,PatientVisitUID,Slopes,L2:BUN,L3:Creatinine,L4:Sodium,L5:Potassium,L6:Chloride,L7:Bicarbonate,L8:Uric,...,L23:LDL Cholesterol,L24:HDL Cholesterol,L25:Parathyroid hormone,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs)
0,2299,5812902,-7.158596,6.5,0.82,143,3.8,103,29.7,NaN,...,129,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2299,6947220,5.550691,6.5,0.88,143,4.3,103,29.7,NaN,...,139,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2299,8186767,-6.063710,14.5,1.10,141,4.4,102,27.2,NaN,...,129,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2299,9385133,-12.812245,15.4,0.72,131,4.2,96,28.2,NaN,...,100,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2304,4916314,28.287500,20.8,1.08,131,4.2,96,28.2,NaN,...,89,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,13.1,9.57,131,4.8,100,18.4,7.3,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52615,3319971,10553916,-4.256753,13.1,1.16,134,4.8,102,16.7,7.3,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52616,3320048,10572126,-0.692078,33.6,0.81,137,3.8,104,16.5,7.3,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52617,3320050,10572565,7.217875,30.7,0.79,140,4.3,105,16.1,7.3,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505


In [8]:
DiagDF = DiagDF.iloc[:, 1:]
DiagDF

,PatientUID,PatientVisitUID,Slopes,D1:Hypertension,D2:Diabetes millitus,D3:Dyslipidemia,D4:SLE,D5:Systemic sclerosis,D6:Rheumatoid arthritis,D7:HIV infection,D8:HBV infection,D9:HCV infection,D10:Ischemic heart disease,D11:Heart failure
0,2299,5812902,-7.158596,0,1,1,0,0,0,0,0,0,0,0
1,2299,6947220,5.550691,0,1,1,0,0,0,0,0,0,0,0
2,2299,8186767,-6.063710,1,1,1,0,0,0,0,0,0,0,0
3,2299,9385133,-12.812245,0,0,1,0,0,0,0,0,0,0,0
4,2304,4916314,28.287500,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,1,0,0,0,0,0,0,0,0,0,0
52615,3319971,10553916,-4.256753,1,0,0,0,0,0,0,0,0,0,0
52616,3320048,10572126,-0.692078,0,0,0,2,0,0,0,0,0,0,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,0,0,0,0


In [9]:
DrugDF = DrugDF.iloc[:, 1:]
DrugDF

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,G1M9:Vasodilator,G1M8:Alpha blocker,G2M6:Insulin,G2M4:TZD,G4M2:Febuxostat,G3M2:Ezetimibe,G1M4:Non-dihydropyridine CCB,G5M1:Ketoanalog amino acid,G2M7:GLP1-RA,G4M3:Uricosuric agent
0,2299,5812902,-7.158596,100,100,20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2299,6947220,5.550691,35,0,14,35,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2299,8186767,-6.063710,100,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2299,9385133,-12.812245,90,0,14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2304,4916314,28.287500,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## **TestMerge**

In [10]:
import pandas as pd

# Assuming you have four dataframes: DrugDF1, DiagDF, DrugDF2, LabDF

# Merge DrugDF1, DiagDF, DrugDF2, and LabDF based on the common columns
merged_df = DrugDF.merge(DiagDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer') \
                    .merge(VitalDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer') \
                    .merge(LabDF, on=['PatientUID', 'PatientVisitUID', 'Slopes'], how='outer')

# The merged_df will contain all the data from the four dataframes, merged on the common columns.
merged_df

,PatientUID,PatientVisitUID,Slopes,G1M2:ARB,G3M1:Statin,G5M2:NSAIDs,G1M5:Diuretic,G1M7:Beta blocker,G1M3:Dihydropyridine CCB,G1M6:MRA,...,L23:LDL Cholesterol,L24:HDL Cholesterol,L25:Parathyroid hormone,L26:Vitamin D,L27:UrineAlbumin/Cr ratio,L28:UrineProtein/Cr ratio,L29:UrineVolume (24hrs),L30:UrineUrea (24hrs),L31:UrineSodium (24hrs),L32:UrineProtein (24hrs)
0,2299,5812902,-7.158596,100,100,20,0,0,0,0,...,129,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2299,6947220,5.550691,35,0,14,35,0,0,0,...,139,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2299,8186767,-6.063710,100,0,1,0,0,0,0,...,129,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2299,9385133,-12.812245,90,0,14,0,0,0,0,...,100,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2304,4916314,28.287500,0,0,1,0,0,0,0,...,89,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52614,3319961,10553903,10.568365,0,0,0,0,0,0,0,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52615,3319971,10553916,-4.256753,0,0,0,0,0,0,0,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52616,3320048,10572126,-0.692078,0,0,0,0,0,0,0,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505
52617,3320050,10572565,7.217875,0,0,0,0,0,0,0,...,62,47,22.90,27.40,13.37,71.1,873,2.3,194.0,505


In [11]:
merged_df.shape

(52619, 72)